## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-23-03-00-00 +0000,world,The New Hot Topic in European Politics Is Air ...,https://www.wsj.com/world/europe/europe-air-co...
1,2025-07-23-03-00-00 +0000,world,Trump’s Fixation With Greenland Has Ended Denm...,https://www.wsj.com/world/europe/trump-denmark...
2,2025-07-23-03-00-00 +0000,world,Inside Syria’s Sectarian Cauldron: A Kidnappin...,https://www.wsj.com/world/middle-east/syria-is...
3,2025-07-22-21-08-00 +0000,world,Protests Break Out as Ukraine Defangs Anticorr...,https://www.wsj.com/world/europe/ukraine-moves...
4,2025-07-22-20-06-00 +0000,world,Opinion | Putin’s Blitz Has United Ukraine,https://www.wsj.com/opinion/putins-blitz-has-u...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2283/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,air,2
25,ukraine,2
49,government,2
59,china,2
37,aid,2


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
7,2025-07-22-19-39-00 +0000,world,Food Aid Becomes Main Sticking Point in Gaza C...,https://www.wsj.com/world/middle-east/food-aid...,11
11,2025-07-22-10-04-00 +0000,world,China’s Xi Gives Up Air Miles for More Time at...,https://www.wsj.com/world/china/chinas-xi-give...,9
10,2025-07-22-10-35-00 +0000,world,A Populist Party That Began on YouTube Helped ...,https://www.wsj.com/world/asia/a-populist-part...,9
0,2025-07-23-03-00-00 +0000,world,The New Hot Topic in European Politics Is Air ...,https://www.wsj.com/world/europe/europe-air-co...,8
2,2025-07-23-03-00-00 +0000,world,Inside Syria’s Sectarian Cauldron: A Kidnappin...,https://www.wsj.com/world/middle-east/syria-is...,8


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
7,11,2025-07-22-19-39-00 +0000,world,Food Aid Becomes Main Sticking Point in Gaza C...,https://www.wsj.com/world/middle-east/food-aid...
10,9,2025-07-22-10-35-00 +0000,world,A Populist Party That Began on YouTube Helped ...,https://www.wsj.com/world/asia/a-populist-part...
11,9,2025-07-22-10-04-00 +0000,world,China’s Xi Gives Up Air Miles for More Time at...,https://www.wsj.com/world/china/chinas-xi-give...
2,8,2025-07-23-03-00-00 +0000,world,Inside Syria’s Sectarian Cauldron: A Kidnappin...,https://www.wsj.com/world/middle-east/syria-is...
3,8,2025-07-22-21-08-00 +0000,world,Protests Break Out as Ukraine Defangs Anticorr...,https://www.wsj.com/world/europe/ukraine-moves...
8,8,2025-07-22-14-59-00 +0000,world,"U.K. Government Will Stick to Budget Rules, Tr...",https://www.wsj.com/world/uk/u-k-government-wi...
1,7,2025-07-23-03-00-00 +0000,world,Trump’s Fixation With Greenland Has Ended Denm...,https://www.wsj.com/world/europe/trump-denmark...
0,6,2025-07-23-03-00-00 +0000,world,The New Hot Topic in European Politics Is Air ...,https://www.wsj.com/world/europe/europe-air-co...
6,4,2025-07-22-19-55-00 +0000,world,Opinion | U.S. Global Health Aid Is an Exempla...,https://www.wsj.com/opinion/u-s-global-health-...
4,3,2025-07-22-20-06-00 +0000,world,Opinion | Putin’s Blitz Has United Ukraine,https://www.wsj.com/opinion/putins-blitz-has-u...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
